# SNP/Phenotype detection from raw text

In [2]:
%load_ext autoreload
%autoreload 2

import sys

In [3]:
# import snorkel and gwasdb
sys.path.append('../snorkel')
sys.path.append('../src')

# set up paths
abstract_dir = '../data/db/papers'

## Assemble a corpus

We will look at abstracts for now

In [4]:
from snorkel.parser import XMLDocParser
from extractor.parser import UnicodeXMLDocParser

xml_parser = UnicodeXMLDocParser(
    path=abstract_dir,
    doc='./*',
    text='.//abstract/p//text()',
    id='.//article-id[@pub-id-type="pmid"]/text()',
    keep_xml_tree=True)

Warning: there are unicode chars that cause trouble, e.g.:

In [23]:
import unicodedata
print u'\u2212 8'
txt=u'\xd7'
print unicodedata.normalize('NFKD', txt).encode('ascii','ignore')
print unicode(txt)

−8

×


Get the actual docs:

In [20]:
import lxml.etree as et
with open('../data/db/papers/19305408.xml') as f:
    for i,doc in enumerate(et.parse(f).xpath('./*')):
        print doc
        print doc.xpath('.//abstract/p//text()')

<Element article at 0x106398ea8>
[u'QT interval duration reflecting myocardial repolarization on the electrocardiogram is a heritable risk factor for sudden cardiac death and drug-induced arrhythmias. We conducted a meta-analysis of 3 genome-wide association studies in 13,685 individuals of European ancestry from the Framingham Heart Study, the Rotterdam Study and the Cardiovascular Health Study. We observed associations at P < 5\xd710', u'\u22128', ' with variants in ', 'NOS1AP', ', ', 'KCNQ1, KCNE1', ', ', 'KCNH2', ' and ', 'SCN5A', ', known to be involved in myocardial repolarization and Mendelian Long QT Syndromes. Associations at five novel loci included 16q21 near ', 'NDRG4', ' and ', 'GINS3', ', 6q22 near ', 'PLN', ', 1p36 near ', 'RNF207', ', 16p13 near ', 'LITAF', ' and 17q12 near ', 'LIG3', ' and ', 'RIFFL', u'. Collectively, the 14 independent variants at these 10 loci explain 5.4\u20136.5% of variation in QT interval. Identifying the causal variants and defining their impac

In [5]:
from snorkel.parser import SentenceParser
from snorkel.parser import Corpus

sent_parser = SentenceParser()
%time corpus = Corpus(xml_parser, sent_parser)

Parsing documents...
Parsing contexts...
Parsed 589 documents and 4799 contexts
CPU times: user 8.48 s, sys: 940 ms, total: 9.42 s
Wall time: 1min 23s


Test the results:

In [8]:
doc = corpus.get_docs()[2]
print doc

Document(id='21298047', file='21298047.xml', text=u'Substance dependence or addiction is a complex environmental and genetic disorder that results in serious health and socio-economic consequences. Multiple substance dependence categories together, rather than any one individual addiction outcome, may explain the genetic variability of such disorder. In our study, we defined a composite substance dependence phenotype derived from six individual diagnoses: addiction to nicotine, alcohol, marijuana, cocaine, opiates or other drugs as a whole. Using data from several genomewide case-control studies, we identified a strong (Odds ratio \u200a=\u200a1.77) and significant (p-value\u200a=\u200a7E-8) association signal with a novel gene, PBX/knotted 1 homeobox 2 (PKNOX2), on chromosome 11 with the composite phenotype in European-origin women. The association signal is not as significant when individual outcomes for addiction are considered, or in males or African-origin population. Our findings

In [9]:
sent = corpus.get_contexts_in(doc.id)[0]
print sent

Sentence(id='21298047-0', doc_id='21298047', doc_name='21298047.xml', sent_id=0, words=[u'Substance', u'dependence', u'or', u'addiction', u'is', u'a', u'complex', u'environmental', u'and', u'genetic', u'disorder', u'that', u'results', u'in', u'serious', u'health', u'and', u'socio-economic', u'consequences', u'.'], lemmas=[u'substance', u'dependence', u'or', u'addiction', u'be', u'a', u'complex', u'environmental', u'and', u'genetic', u'disorder', u'that', u'result', u'in', u'serious', u'health', u'and', u'socio-economic', u'consequence', u'.'], poses=[u'NN', u'NN', u'CC', u'NN', u'VBZ', u'DT', u'JJ', u'JJ', u'CC', u'JJ', u'NN', u'WDT', u'VBZ', u'IN', u'JJ', u'NN', u'CC', u'JJ', u'NNS', u'.'], dep_parents=[2, 11, 2, 2, 11, 11, 11, 11, 8, 8, 0, 13, 11, 16, 16, 13, 16, 19, 16, 11], dep_labels=[u'compound', u'nsubj', u'cc', u'conj', u'cop', u'det', u'amod', u'amod', u'cc', u'conj', u'ROOT', u'nsubj', u'acl:relcl', u'case', u'amod', u'nmod', u'cc', u'amod', u'conj', u'punct'], char_offsets=[

## Try extracting rs-ids first